In [1]:
import keras
from keras import models
from keras import optimizers
from keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping

import datetime

callback = keras.callbacks.EarlyStopping(monitor='loss',patience=3)

2024-05-05 14:05:19.237969: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
%store -r EPOCH
%store -r STEPS_PER_EPOCH
%store -r VALIDATION_STEPS

%store -r X_train
%store -r y_train
%store -r X_test 
%store -r y_test 

%store -r train_dir 
%store -r test_dir
%store -r model_dir

%store -r class_names
%store -r class_numbers

```
layers.Conv2D(
    filters: int, the dimension of the output space (the number of filters in the convolution).kernel_size: int or tuple/list of 2 integer, specifying the size of the convolution window.
)
```

In [3]:
def plot_acc_loss(x):  
  acc = x.history["acc"]
  val_acc = x.history["val_acc"]
  loss = x.history["loss"]
  val_loss = x.history["val_loss"]
  
  print("acc =", acc[-1])
  print("val_acc = ",val_acc[-1])
  print("loss =", loss[-1])
  print("val_loss =", val_loss[-1])
  epochs = range(1, len(acc) + 1)
  fig=plt.figure()
  plt.subplot(2,1,1)
  plt.plot(epochs, acc, "bo", label="Training acc")
  plt.plot(epochs, val_acc, "b", label="Validation acc")
  plt.xlabel('Epoch')
  plt.ylabel('Acc')
  plt.title("Training and Validation Accuracy")

  plt.subplot(2,1,2)
  plt.plot(epochs, loss, "bo", label="Training loss")
  plt.plot(epochs, val_loss, "b", label="Validation loss")
  plt.title("Training and Validation Loss")
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  fig.tight_layout()
  plt.show()

In [4]:
def create_model():
  # Define model
  model = models.Sequential()

  # Layer 1
  model.add(
    layers.Conv2D(
      64,(3,3),
      padding='same',
      activation='relu',
      input_shape=(32,32,3)
      )
  )

  model.add(layers.MaxPooling2D((2, 2)))

  # Layer 3
  model.add(
    layers.Conv2D(
      64,(3, 3), 
      padding='same',
      activation='relu'
    )
  )

  model.add(layers.MaxPooling2D((2, 2)))

  # Layer 4
  model.add(
    layers.Conv2D(
      128,(3,3),
      padding='same',
      activation='relu'
    )
  )

  model.add(layers.MaxPooling2D((2, 2)))

# Layer 5
  model.add(
    layers.Conv2D(
      128,(3,3),
      padding='same',
      activation='relu'
    )
  )

  # Dense layer:
  model.add(layers.Flatten())
  model.add(layers.Dropout(0.50))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(0.50))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(len(class_names), activation='softmax'))

  # Compile
  model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adamax(lr=5e-3), metrics=['acc'])

  return model

In [5]:
# Get Model Summary
def model_summary(model):
    model.summary()

In [6]:
def fit_model(model):
    train_datagen=ImageDataGenerator(rescale=1./255)
    test_datagen= ImageDataGenerator(rescale=1./255)

    #veri zenginleştirilmesi yapılıyor overfitting engellemek için
    datagen = ImageDataGenerator(
                zoom_range=0.2,
                horizontal_flip=True,
                rescale=1./255)

    #Train:
    train_generator = datagen.flow_from_directory(train_dir,
                                            target_size = (32,32),
                                            batch_size =20,
                                            class_mode = 'categorical')

    test_generator = datagen.flow_from_directory(test_dir,
                                            target_size = (32,32),
                                            batch_size = 20,
                                            class_mode = 'categorical')

    history = model.fit(
        train_generator,
        callbacks=callback,
        steps_per_epoch=STEPS_PER_EPOCH,                              
        epochs=EPOCH,
        validation_data=test_generator,
        validation_steps=VALIDATION_STEPS
    ) 

    plot_acc_loss(history)

    current_time = datetime.datetime.now()

    # Format the date and time
    formatted_time = current_time.strftime("%Y-%m-%d_%H:%M:%S")

    model.save(model_dir + 'cifar100_' + formatted_time + '.h5')
    model.save('cifar100.h5')

    return history